# Les APIs REST avec Python

## Création API REST avec Flask

### Installation
Transformer la cellule Markdown en cellule de code pour exécuter la partie qui correspond à votre setup

```python
# Pipy
!pip install Flask
# Conda
!conda install -c anaconda flask
```

Exemple minimaliste

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

#app.run()

### Configuration de Flask
Différentes manières de configurer, paramètres basiques pour l'instant

In [ ]:
# Méthode 1: Configuration directe
app.config['DEBUG'] = True
app.config['SECRET_KEY'] = 'un_secret_très_sécurisé'

# Méthode 2: Configuration à partir d'une classe séparée
class Config(object):
    DEBUG = True
    SECRET_KEY = 'un_secret_très_sécurisé'

app.config.from_object(Config)

# Méthode 3: Configuration à partir d'un fichier CFG
app.config.from_pyfile('config/config.cfg')

# Méthode 4: Configuration à partir de variables d'environnement
import os
app.config['DEBUG'] = os.environ.get('DEBUG', False)
app.config['SECRET_KEY'] = os.environ.get('SECRET_KEY', 'un_secret_très_sécurisé')

# Méthode 5: Configuration à partir d'un fichier JSON
import json
with open('config/config.json') as f:
    app.config.update(json.load(f))

### Définition des routes
Ajout de routes basiques

In [29]:
# Route basique
@app.route('/login')
def login():
    return 'login'

# Route avec paramètre (variable)
@app.route('/user/<username>')
def profile(username):
    return f'{username}\'s profile'

# Route avec paramètre converti / casté (en int)
@app.route('/post/<int:post_id>')
def show_post(post_id):
    return f'Post {post_id}'

### Définition des méthodes
Création des fonctions à l'aide des décorateurs de route

In [ ]:
# GET
@app.route('/get_example')
def get_example():
    return "Ceci est une réponse de GET!"

# POST
from flask import request
@app.route('/post_example', methods=['POST'])
def post_example():
    # Traitement des données envoyées (données brutes)
    return request.args.get('keyword')

# GET et POST (classique pour un formulaire)
@app.route('/multi_method', methods=['GET', 'POST'])
def multi_method():
    if request.method == 'POST':
        # Traitement des données envoyées (données de formulaire)
        return request.form['name']
    else:
        # GET par défaut
        return "Renvoi du formulaire"

# PUT
@app.route('/update_item', methods=['PUT'])
def update_item():
    # Code pour mettre à jour un élément
    return "Élément mis à jour"

# DELETE
@app.route('/delete_item', methods=['DELETE'])
def delete_item():
    # Code pour supprimer un élément
    return "Élément supprimé"


### Définition des réponses

In [ ]:
# Renvoi d'une réponse simple
@app.route('/')
def home():
    return "Bienvenue sur ma page d'accueil!"

# Utilisation de make_response pratique pour ajouter 
# des éléments (code, headers, etc.) à une réponse pré-existante 
from flask import make_response
@app.route('/custom')
def custom_response():
    resp = make_response("Contenu personnalisé", 202)
    resp.headers['X-Something'] = 'Valeur'
    return resp

# Renvoi d'un objet Response avec la classe Response
# Permet un contrôle direct et complet sur les données
# de réponse, statut HTTP, en-têtes, cookies, etc.
from flask import Response
@app.route('/response')
def response_example():
    return Response("Contenu complet", status=200, headers={'Content-Type': 'text/plain'})

# Renvoi d'un JSON
from flask import jsonify
@app.route('/json')
def json_example():
    return jsonify({'key': 'value', 'listKey': [1, 2, 3]})

# Utilisation de templates (pages HTML)
# Nécessaire pour créer un site web
from flask import render_template
@app.route('/template')
def template_example():
    return render_template('template.html', variable='valeur')

# Gestions et renvoi de messages d'erreurs
@app.errorhandler(404)
def page_not_found(error):
    return "Cette page n'a pas été trouvée", 404

# Streaming
from flask import stream_with_context
@app.route('/stream')
def stream_example():
    def generate():
        yield 'Hello '
        yield 'World!'
    return Response(stream_with_context(generate()))

# Redirection vers une autre page
from flask import redirect
@app.route('/redirect')
def redirect_example():
    return redirect('/')

# Redirection vers une autre page avec url_for
# Permet de ne pas avoir à écrire l'URL en dur
# Nécessite d'avoir défini les routes avec un nom
# Exemple: app.route('/', name='home')
# Nécessaire pour créer un site web
from flask import url_for
@app.route('/redirect')
def redirect_example():
    return redirect(url_for('home'))


## Consommation des APIs REST avec Python

### Utilisation de la bibliothèque requests

In [1]:
# Communiquer avec l'API "example_3_basic_api_items.py"
import requests

In [28]:
# POST
item_name = 'tomate'
new_item = requests.post(f'http://localhost:5000/item/{item_name}', json={item_name: 3})
print(new_item)
print(new_item.json())

<Response [201]>
{'data': {'tomate': 3}, 'name': 'tomate'}


In [29]:
# GET
item_1 = requests.get(f'http://localhost:5000/item/{item_name}')
print(item_1)
try:
    print(item_1.json())
except:
    print('not found')

<Response [200]>
{'data': {'tomate': 3}, 'name': 'tomate'}


In [18]:
# PUT
update_item = requests.put('http://localhost:5000/item/courgette', json={'courgette': 300})
print(update_item)
print(update_item.json())

<Response [200]>
{'data': {'courgette': 300}, 'name': 'courgette'}


In [30]:
# GET sur tous les items
items = requests.get('http://localhost:5000/items')
print(items)
print(items.json())

<Response [200]>
{'patate': {'data': {'patate': 10}, 'name': 'patate'}, 'tomate': {'data': {'tomate': 3}, 'name': 'tomate'}}


### Gestion des réponses

### Gestion des erreurs

## Cyber problèmes

### SQL Injection et Cross-Site Scripting (XSS)

Solution: utiliser `escape`

In [ ]:
# Puisque l'utilisateur peut entrer n'importe quoi dans l'URL
# il est important de filtrer les paramètres avec 'escape'

from markupsafe import escape
@app.route("/<name>")
def hello(name):
    return f"Hello, {escape(name)}!"

Ce qu'il ne faut pas faire

In [ ]:
@app.route('/search', methods=['GET'])
def search():
    user_name = request.args.get('username')
    query = "SELECT email FROM users WHERE name = '" + user_name + "'"  # Non sécurisé
    cursor = get_db().execute(query)
    result = cursor.fetchone()
    cursor.close()
    if result:
        return "L'adresse email de l'utilisateur est : " + result[0]
    else:
        return "Utilisateur non trouvé"

L'attaquant peut simplement entrer dans l'URL:
```bash
http://127.0.0.1:5000/search?username=' OR '1'='1
```

La requête serait donc interprétée en:
```sql
SELECT email FROM users WHERE name = '' OR '1'='1'
```

Comme la condition '1'='1' est toujours vraie, la requête renverra les adresses mail **de tous les utilisateurs** de la base de données

Ce qu'il faut faire

In [ ]:
@app.route('/search', methods=['GET'])
def search_secure():
    user_name = request.args.get('username')
    query = "SELECT email FROM users WHERE name = ?"
    cursor = get_db().execute(query, (user_name,))
    result = cursor.fetchone()
    cursor.close()
    if result:
        return "L'adresse email de l'utilisateur est : " + result[0]
    else:
        return "Utilisateur non trouvé"

Flask et son moteur de templating Jinja2 appliquent automatiquement **l'échappement des caractères** pour prévenir les attaques XSS (et injection SQL). Quand vous utilisez {{ variable }} dans un template Jinja2, Flask **convertit automatiquement** tous les **caractères dangereux** en entités **HTML**. Cela empêche l'interprétation de ces caractères comme du code lorsqu'ils sont affichés dans le navigateur de l'utilisateur.

## Mise à disposition

Mise à disposition pour l'ensemble des appareils sur le réseau
```
flask --app hello_world run --host=0.0.0.0
flask --app hello run --debug
```